In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
paysim = pd.read_csv("/home/ayush/Desktop/research papers/minor/paysim.csv")

In [29]:
# proof of imbalanced datasets by using isFraud feature
print("Number of correct transactions are {} \nNumber of fraud transactions are {} ".\
       format(paysim.loc[paysim["isFraud"]==0,"step"].count(),paysim.loc[paysim["isFraud"]==1,"step"].count()))

Number of correct transactions are 6354407 
Number of fraud transactions are 8213 


In [37]:
#1.1  checking in which types frauds occured
print('\n The types of fraudulent transactions are {}'.format(\
list(paysim.loc[paysim.isFraud == 1].type.drop_duplicates().values)))

# getting number of fraudulent transactions in transfers and cashouts
FraudTransfer = paysim.loc[(paysim.isFraud == 1) & (paysim.type == 'TRANSFER')]
FraudCashout = paysim.loc[(paysim.isFraud == 1) & (paysim.type == 'CASH_OUT')]

print ('\n The number of fraudulent TRANSFERs = {}'.\
       format(len(FraudTransfer)))

print ('\n The number of fraudulent CASH_OUTs = {}'.\
       format(len(FraudCashout))) 


 The types of fraudulent transactions are ['TRANSFER', 'CASH_OUT']

 The number of fraudulent TRANSFERs = 4097

 The number of fraudulent CASH_OUTs = 4116


In [38]:
#1.2 The data is described as isFlaggedFraud being set when an attempt is made to 'TRANSFER' an 'amount' greater than 2,00,000. 
#In fact, as shown below, isFlaggedFraud can remain not set despite this condition being met.
print('\nThe type of transactions in which isFlaggedFraud is set: \
{}'.format(list(paysim.loc[paysim.isFlaggedFraud == 1].type.drop_duplicates()))) 
                                                            

Transfer = paysim.loc[paysim.type == 'TRANSFER']
Flagged = paysim.loc[paysim.isFlaggedFraud == 1]
NotFlagged = paysim.loc[paysim.isFlaggedFraud == 0]

print('\nMin amount transacted when isFlaggedFraud is set= {}'\
                                  .format(Flagged.amount.min())) # 353874.22

print('\nMax amount transacted in a TRANSFER where isFlaggedFraud is not set=\
 {}'.format(Transfer.loc[Transfer.isFlaggedFraud == 0].amount.max())) # 92445516.64


The type of transactions in which isFlaggedFraud is set: ['TRANSFER']

Min amount transacted when isFlaggedFraud is set= 353874.22

Max amount transacted in a TRANSFER where isFlaggedFraud is not set= 92445516.64


In [42]:
#The number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and newBalanceDest = 0
print('\nThe number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and\
 newBalanceDest = 0: {}'.\
format(len(Transfer.loc[(Transfer.isFlaggedFraud == 0) & \
(Transfer.oldbalanceDest == 0) & (Transfer.newbalanceDest == 0)]))) 


The number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and newBalanceDest = 0: 4158


In [44]:
# seeeing if isFlaggedFraud has any correlation with oldBalance Original and newbalance original or not 
print('\nMin, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: {}'.\
format([round(Flagged.oldbalanceOrg.min()), round(Flagged.oldbalanceOrg.max())]))

print('\nMin, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where \
oldBalanceOrig = \
newBalanceOrig: {}'.format(\
[Transfer.loc[(Transfer.isFlaggedFraud == 0) & (Transfer.oldbalanceOrg \
== Transfer.newbalanceOrig)].oldbalanceOrg.min(), \
round(Transfer.loc[(Transfer.isFlaggedFraud == 0) & (Transfer.oldbalanceOrg \
               == Transfer.newbalanceOrig)].oldbalanceOrg.max())]))


Min, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: [353874.0, 19585040.0]

Min, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where oldBalanceOrig = newBalanceOrig: [0.0, 575668.0]


In [45]:
# seeing if isFlaggedFraud be set based on seeing a customer transacting more than once 
print('\nHave originators of transactions flagged as fraud transacted more than \
once? {}'\
.format((Flagged.nameOrig.isin(pd.concat([NotFlagged.nameOrig, \
                                    NotFlagged.nameDest]))).any())) # False

print('\nHave destinations for transactions flagged as fraud initiated\
 other transactions? \
{}'.format((Flagged.nameDest.isin(NotFlagged.nameOrig)).any())) # False

# Since only 2 destination accounts of 16 that have 'isFlaggedFraud' set have been
# destination accounts more than once,
# clearly 'isFlaggedFraud' being set is independent of whether a 
# destination account has been used before or not

print('\nHow many destination accounts of transactions flagged as fraud have been \
destination accounts more than once?: {}'\
.format(sum(Flagged.nameDest.isin(NotFlagged.nameDest))))


Have originators of transactions flagged as fraud transacted more than once? False

Have destinations for transactions flagged as fraud initiated other transactions? False

How many destination accounts of transactions flagged as fraud have been destination accounts more than once?: 2


In [46]:
#1.3 
#Are expected merchant accounts accordingly labelled?
# proving that CASH_IN doesnt involve paying by a merchant.
print('\nAre there any merchants among originator accounts for CASH_IN \
transactions? {}'.format(\
(paysim.loc[paysim.type == 'CASH_IN'].nameOrig.str.contains('M')).any())) # False


Are there any merchants among originator accounts for CASH_IN transactions? False


In [47]:
# proving that CASH_OUT are doesnt involve paying to merchants
print('\nAre there any merchants among destination accounts for CASH_OUT \
transactions? {}'.format(\
(paysim.loc[paysim.type == 'CASH_OUT'].nameDest.str.contains('M')).any())) # False


Are there any merchants among destination accounts for CASH_OUT transactions? False


In [48]:
# proving that  there are no merchants among any originator accounts. 
# Merchants are only present in destination accounts for all PAYMENTS.
print('\nAre there merchants among any originator accounts? {}'.format(\
      paysim.nameOrig.str.contains('M').any())) # False

print('\nAre there any transactions having merchants among destination accounts\
 other than the PAYMENT type? {}'.format(\
(paysim.loc[paysim.nameDest.str.contains('M')].type != 'PAYMENT').any())) # False


Are there merchants among any originator accounts? False

Are there any transactions having merchants among destination accounts other than the PAYMENT type? False


In [49]:
# 1.4 
#within this two-step process, the fraudulent account would be both, the destination in 
# a TRANSFER and the originator in a CASH_OUT.However, the data shows below that there are no such common 
# accounts among fraudulent transactions. 
print('\nWithin fraudulent transactions, are there destinations for TRANSFERS \
that are also originators for CASH_OUTs? {}'.format(\
(FraudTransfer.nameDest.isin(FraudCashout.nameOrig)).any())) # False
NotFraud = paysim.loc[paysim.isFraud == 0]


Within fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? False


In [50]:
#Number of destination accounts for fraudulent TRANSFERs originate CASHOUTs that are not detected and are labeled as genuine? 
print('\nFraudulent TRANSFERs whose destination accounts are originators of \
genuine CASH_OUTs: \n\n{}'.format(FraudTransfer.loc[FraudTransfer.nameDest.\
isin(NotFraud.loc[NotFraud.type == 'CASH_OUT'].nameOrig.drop_duplicates())])) #3



Fraudulent TRANSFERs whose destination accounts are originators of genuine CASH_OUTs: 

         step      type      amount     nameOrig  oldbalanceOrg  \
1030443    65  TRANSFER  1282971.57  C1175896731     1282971.57   
6039814   486  TRANSFER   214793.32  C2140495649      214793.32   
6362556   738  TRANSFER   814689.88  C2029041842      814689.88   

         newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  isFraud  \
1030443             0.0  C1714931087             0.0             0.0        1   
6039814             0.0   C423543548             0.0             0.0        1   
6362556             0.0  C1023330867             0.0             0.0        1   

         isFlaggedFraud  
1030443               0  
6039814               0  
6362556               0  
